# **Dynamic Tool Calling with Decorators and Function Introspection**

In this notebook, we will explore how to create a simple system where functions can be registered as tools and dynamically invoked based on user input. We will leverage Python concepts like **decorators** and **function introspection** using the `inspect` module.

### **Objectives**:
- Learn how to use decorators to register functions as tools.
- Understand how to extract function metadata (like parameters and docstrings) using Python's `inspect` module.
- Dynamically invoke functions (tools) using their names and arguments.

### **1. Introduction to Decorators**

Decorators are a powerful feature in Python that allow you to modify or extend the behavior of functions or methods without changing their actual code.

Let's start by reviewing how decorators work with a simple example.

In [1]:
def simple_decorator(func):
    def wrapper(*args, **kwargs):
        print("Before the function call")
        result = func(*args, **kwargs)
        print("After the function call")
        return result
    return wrapper

@simple_decorator
def greet(name):
    print(f"Hello, {name}!")

# Call the decorated function
greet("Alice")

Before the function call
Hello, Alice!
After the function call


### **Explanation**:
- The `@simple_decorator` modifies the behavior of `greet`. Before and after calling `greet`, additional print statements are executed.

---

### **2. Registering Functions as Tools**

We can use decorators to mark functions as special "tools". In our case, we want to register functions as tools so we can dynamically list and invoke them later.

We'll create a `ToolMixin` class that manages tool registration and invocation.

In [2]:
import inspect
import functools
from typing import Callable, Any, List, Dict

# The main class to manage tool registration and invocation
class ToolMixin:
    """
    ToolMixin allows registering functions as tools with metadata such as docstring (description)
    and input parameters.
    """
    _tools: Dict[str, dict] = {}  # A class-level dictionary to store registered tools

    @classmethod
    def register_tool(cls, func: Callable):
        """
        Registers a function as a tool using its docstring as a description.
        """
        tool_name = func.__name__
        # Extract the function's signature (parameter names) and docstring
        sig = inspect.signature(func)
        description = inspect.getdoc(func)  # Automatically fetches the function's docstring

        # Store the function metadata
        cls._tools[tool_name] = {
            "function": func,
            "description": description,
            "parameters": sig.parameters
        }

    @classmethod
    def get_tool(cls, name: str) -> Callable:
        """
        Retrieves the tool (function) by name.
        """
        if name in cls._tools:
            return cls._tools[name]["function"]
        raise ValueError(f"Tool '{name}' not found.")

    @classmethod
    def list_tools(cls) -> List[Dict[str, Any]]:
        """
        Returns a list of all registered tools and their metadata (docstring and parameters).
        """
        return [
            {"name": name, "description": data["description"], "parameters": data["parameters"]}
            for name, data in cls._tools.items()
        ]

    def invoke_tool(self, tool_name: str, **kwargs):
        """
        Invokes a registered tool with the given keyword arguments.
        """
        if tool_name in self._tools:
            tool = self._tools[tool_name]["function"]
            return tool(self, **kwargs)  # Call the tool function
        raise ValueError(f"Tool '{tool_name}' not found.")

---

### **3. Using a Decorator to Register Tools**

We can now create a `tool` decorator that marks functions as tools and registers them automatically with our `ToolMixin`.

In [3]:
def tool(func: Callable):
    """
    A decorator that marks a method as a tool, registering it with metadata such as
    docstring and parameter names.
    """
    # Register the tool in the ToolMixin class
    @functools.wraps(func)
    def wrapper(self, *args, **kwargs):
        return func(*args, **kwargs)
    
    ToolMixin.register_tool(func)  # Register the tool
    return wrapper

---

### **4. Defining Tools with the `tool` Decorator**

We can now define tools in our `Assistant` class and register them using the `@tool` decorator.

In [4]:
# Assistant class inheriting from ToolMixin to register and invoke tools
class Assistant(ToolMixin):
    """
    Assistant class that uses the ToolMixin to register and invoke tools.
    """

    @tool
    def get_current_temperature(self, location: str, unit: str) -> str:
        """
        Get the current temperature in a specified location.
        """
        return f"Temperature in {location} is 72 {unit}"

    @tool
    def get_rain_probability(self, location: str) -> str:
        """
        Get the probability of rain for a given location.
        """
        return f"Probability of rain in {location} is 15%."

---

### **5. Listing and Invoking Tools**

Now that we have registered tools in our `Assistant` class, let's list the available tools and invoke them dynamically based on their names.

In [5]:
# Initialize the assistant
assistant = Assistant()

# List available tools
print("Available Tools:")
for tool_info in assistant.list_tools():
    print(f"Tool: {tool_info['name']}")
    print(f"  Description: {tool_info['description']}")
    print(f"  Parameters: {tool_info['parameters']}")
    print()

# Invoke tools dynamically
result = assistant.invoke_tool("get_current_temperature", location="New York", unit="Fahrenheit")
print(f"Result from 'get_current_temperature': {result}")

result = assistant.invoke_tool("get_rain_probability", location="San Francisco")
print(f"Result from 'get_rain_probability': {result}")

Available Tools:
Tool: get_current_temperature
  Description: Get the current temperature in a specified location.
  Parameters: OrderedDict({'self': <Parameter "self">, 'location': <Parameter "location: str">, 'unit': <Parameter "unit: str">})

Tool: get_rain_probability
  Description: Get the probability of rain for a given location.
  Parameters: OrderedDict({'self': <Parameter "self">, 'location': <Parameter "location: str">})

Result from 'get_current_temperature': Temperature in New York is 72 Fahrenheit
Result from 'get_rain_probability': Probability of rain in San Francisco is 15%.


---

### **6. Key Concepts Recap**

- **Decorators**: We used decorators to register functions as tools dynamically. The `@tool` decorator adds metadata about the function without modifying its core logic.
  
- **Function Introspection**: Using the `inspect` module, we extracted metadata like the function signature and docstring, which provided us with details about the parameters and purpose of the tools.
  
- **Dynamic Function Invocation**: We dynamically invoked functions by looking them up based on their name. This allowed us to create a system where tools can be selected and executed at runtime.

---

### **7. Next Steps**

You now have a foundation for building dynamic tool-based systems in Python. Here are a few ways to extend this:
- **Add More Tools**: Register more tools and try invoking them with different arguments.
- **Error Handling**: Add validation to check if the right arguments are passed to the tools.
- **Advanced Features**: You can add categorization, logging, or other metadata to make your tool system even more robust.

---

### **Conclusion**

In this notebook, you learned how to:
- Use **decorators** to register functions dynamically.
- Extract function metadata using **introspection**.
- Invoke functions dynamically based on their names.

This technique can be very useful when building dynamic systems like tool-based assistants, workflow engines, or plugins for applications.